# CitiBike Weather Analysis 2024

This notebook analyzes CitiBike trip data in relation to weather patterns for 2024.

## Objectives:
- Create time series plots of temperature data
- Merge trip count data with weather data
- Create dual-axis charts for trips and temperature
- Analyze trip duration distributions
- Visualize user demographics

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette('husl')

In [ ]:
# Load the weather data
weather_df = pd.read_csv('/Users/Glebazzz/Jupiter/New York\'s CitiBike trips in 2022./weather_data_2024_enhanced.csv')

# Convert date column to datetime
weather_df['date'] = pd.to_datetime(weather_df['date'])

# Display basic info about the dataset
print("Weather Dataset Info:")
print(f"Shape: {weather_df.shape}")
print(f"Date range: {weather_df['date'].min()} to {weather_df['date'].max()}")
print("\nFirst few rows:")
weather_df.head()

## 1. Temperature Time Series Analysis

Let's create a line plot showing temperature variations throughout 2024.

In [ ]:
# Create temperature time series plot using pandas plotting
fig, ax = plt.subplots(figsize=(15, 8))

# Plot temperature data using pandas plotting function
weather_df.set_index('date')[['temp_max', 'temp_min', 'temp_mean']].plot(
    ax=ax,
    linewidth=2,
    alpha=0.8
)

# Customize the plot
ax.set_title('Temperature Variations in 2024', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)
ax.legend(['Max Temperature', 'Min Temperature', 'Mean Temperature'], loc='upper right')
ax.grid(True, alpha=0.3)

# Add seasonal shading
ax.axvspan(pd.to_datetime('2024-03-20'), pd.to_datetime('2024-06-20'), alpha=0.1, color='green', label='Spring')
ax.axvspan(pd.to_datetime('2024-06-20'), pd.to_datetime('2024-09-22'), alpha=0.1, color='orange', label='Summer')
ax.axvspan(pd.to_datetime('2024-09-22'), pd.to_datetime('2024-12-21'), alpha=0.1, color='brown', label='Fall')

plt.tight_layout()
plt.show()

## 2. Simulated Trip Data and Merging

Since we don't have the actual trip data file, I'll create simulated trip counts that correlate with weather patterns.

In [ ]:
# Create simulated trip data that correlates with weather
np.random.seed(42)

# Base trip count influenced by temperature and weather conditions
base_trips = 1000
temp_factor = (weather_df['temp_mean'] - weather_df['temp_mean'].min()) / (weather_df['temp_mean'].max() - weather_df['temp_mean'].min())
weather_factor = np.where(weather_df['total_precipitation'] > 5, 0.6, 1.0)  # Reduce trips on rainy days
seasonal_factor = 1 + 0.5 * np.sin(2 * np.pi * weather_df.index / 365)  # Seasonal variation

# Generate trip counts
trip_counts = (base_trips * (0.5 + temp_factor) * weather_factor * seasonal_factor + 
               np.random.normal(0, 100, len(weather_df))).astype(int)

# Ensure no negative trip counts
trip_counts = np.maximum(trip_counts, 50)

# Add trip counts to weather dataframe
weather_df['trip_count'] = trip_counts

print(f"Trip count statistics:")
print(f"Mean: {trip_counts.mean():.0f}")
print(f"Min: {trip_counts.min()}")
print(f"Max: {trip_counts.max()}")

weather_df[['date', 'temp_mean', 'total_precipitation', 'trip_count']].head(10)

## 3. Dual-Axis Chart: Trip Counts and Temperature

Now let's create a dual-axis chart showing both trip counts and temperature over time.

In [ ]:
# Create dual-axis chart
fig, ax1 = plt.subplots(figsize=(16, 8))

# Plot trip counts on primary y-axis
color1 = 'tab:blue'
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Daily Trip Count', color=color1, fontsize=12)
line1 = ax1.plot(weather_df['date'], weather_df['trip_count'], color=color1, linewidth=2, alpha=0.7, label='Trip Count')
ax1.tick_params(axis='y', labelcolor=color1)

# Create secondary y-axis for temperature
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel('Temperature (°C)', color=color2, fontsize=12)
line2 = ax2.plot(weather_df['date'], weather_df['temp_mean'], color=color2, linewidth=2, alpha=0.8, label='Mean Temperature')
ax2.tick_params(axis='y', labelcolor=color2)

# Add title and grid
ax1.set_title('CitiBike Trip Counts vs Temperature (2024)', fontsize=16, fontweight='bold', pad=20)
ax1.grid(True, alpha=0.3)

# Add combined legend
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')

plt.tight_layout()
plt.show()

### Explanation of Matplotlib Usage

In the above visualizations, I used **both paradigms** of Matplotlib:

1. **Pandas Integration**: For the temperature time series, I used pandas' built-in plotting functionality (`df.plot()`), which provides a convenient interface for quick visualizations while still leveraging matplotlib under the hood.

2. **Object-Oriented (OO) Approach**: For the dual-axis chart, I used matplotlib's OO interface with explicit figure and axes objects (`fig, ax = plt.subplots()`). This approach provides more control over the plot elements and is essential for complex visualizations like dual-axis charts.

**Key techniques used:**
- `twinx()` to create a secondary y-axis sharing the same x-axis
- Explicit color coding for each axis to maintain clarity
- Combined legend handling for multiple axes
- Grid and styling customization for better readability

## 4. Trip Duration Analysis

Let's create simulated trip duration data and analyze it with a histogram and fitted curve.

In [ ]:
# Generate simulated trip duration data (in minutes)
# Using log-normal distribution which is typical for trip durations
np.random.seed(42)
n_trips = 10000

# Log-normal distribution parameters
mu, sigma = 2.5, 0.8
tripduration = np.random.lognormal(mu, sigma, n_trips)

# Convert to minutes and cap at reasonable maximum
tripduration = np.clip(tripduration, 1, 120)  # 1 minute to 2 hours

print(f"Trip duration statistics:")
print(f"Mean: {tripduration.mean():.1f} minutes")
print(f"Median: {np.median(tripduration):.1f} minutes")
print(f"95th percentile: {np.percentile(tripduration, 95):.1f} minutes")

In [ ]:
# Create histogram with fitted curve
fig, ax = plt.subplots(figsize=(12, 8))

# Create histogram
n_bins = 50
counts, bins, patches = ax.hist(tripduration, bins=n_bins, density=True, alpha=0.7, 
                                color='skyblue', edgecolor='black', linewidth=0.5)

# Fit a curve to the data using kernel density estimation
from scipy.stats import gaussian_kde
kde = gaussian_kde(tripduration)
x_range = np.linspace(tripduration.min(), tripduration.max(), 300)
kde_values = kde(x_range)

# Plot the fitted curve
ax.plot(x_range, kde_values, 'r-', linewidth=3, label='KDE Curve', alpha=0.8)

# Alternative: Fit log-normal distribution
from scipy.stats import lognorm
shape, loc, scale = lognorm.fit(tripduration)
x_fitted = np.linspace(tripduration.min(), tripduration.max(), 300)
pdf_fitted = lognorm.pdf(x_fitted, shape, loc, scale)
ax.plot(x_fitted, pdf_fitted, 'g--', linewidth=2, label='Log-Normal Fit', alpha=0.8)

# Customize the plot
ax.set_title('Distribution of Trip Duration with Fitted Curves', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Trip Duration (minutes)', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# Add statistics text box
stats_text = f"Mean: {tripduration.mean():.1f} min\nMedian: {np.median(tripduration):.1f} min\nStd: {tripduration.std():.1f} min"
ax.text(0.75, 0.85, stats_text, transform=ax.transAxes, fontsize=10,
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

## 5. User Demographics Analysis

Let's create simulated user type and gender data, then visualize them using subplots with bar and pie charts.

In [ ]:
# Generate simulated user demographic data
np.random.seed(42)

# User types: Member (70%), Casual (30%)
usertype = np.random.choice(['Member', 'Casual'], size=n_trips, p=[0.7, 0.3])

# Gender: Male (60%), Female (35%), Other/Unknown (5%)
gender = np.random.choice(['Male', 'Female', 'Other'], size=n_trips, p=[0.6, 0.35, 0.05])

# Create summary statistics
usertype_counts = pd.Series(usertype).value_counts()
gender_counts = pd.Series(gender).value_counts()

print("User Type Distribution:")
print(usertype_counts)
print("\nGender Distribution:")
print(gender_counts)

In [ ]:
# Create figure with two subplots using OO approach
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Subplot 1: Bar chart for user types
bars = ax1.bar(usertype_counts.index, usertype_counts.values, 
               color=['#FF6B6B', '#4ECDC4'], alpha=0.8, edgecolor='black', linewidth=1)

# Customize bar chart
ax1.set_title('User Type Distribution', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('User Type', fontsize=12)
ax1.set_ylabel('Number of Trips', fontsize=12)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'{int(height):,}', ha='center', va='bottom', fontweight='bold')

# Subplot 2: Pie chart for gender
colors = ['#FF9999', '#66B2FF', '#99FF99']
wedges, texts, autotexts = ax2.pie(gender_counts.values, labels=gender_counts.index, 
                                   autopct='%1.1f%%', startangle=90, colors=colors,
                                   explode=(0.05, 0.05, 0.05), shadow=True)

# Customize pie chart
ax2.set_title('Gender Distribution', fontsize=14, fontweight='bold', pad=15)

# Enhance text appearance
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

# Add a legend to the pie chart
ax2.legend(wedges, [f'{label}: {count:,}' for label, count in zip(gender_counts.index, gender_counts.values)],
          title="Gender Counts",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.show()

### Explanation of Subplot Implementation

For the demographic analysis, I used matplotlib's **Object-Oriented (OO) approach** with subplots:

**Key techniques:**

1. **Figure and Axes Creation**: Used `plt.subplots(1, 2)` to create a figure with two side-by-side axes (not dual-axis, but separate subplot axes)

2. **Individual Axis Control**: Each subplot (`ax1`, `ax2`) was customized independently:
   - `ax1`: Bar chart with custom colors, value labels, and grid
   - `ax2`: Pie chart with explosion effect, custom colors, and external legend

3. **Advanced Customization**:
   - Added value labels on bar chart using text positioning
   - Used exploded pie chart with shadow effects
   - Positioned legend outside the pie chart area
   - Applied consistent styling across both subplots

This approach demonstrates the power of matplotlib's OO interface for creating complex, multi-panel visualizations where each panel can have completely different chart types and styling.

## 6. Summary and Insights

This analysis demonstrates several key visualization techniques:

### Matplotlib Paradigms Used:

1. **Pandas Integration**: Quick and convenient plotting for exploratory analysis
2. **Object-Oriented Approach**: Full control for complex visualizations

### Key Findings (based on simulated data):

- **Seasonal Patterns**: Trip counts show clear seasonal variation, peaking in warmer months
- **Weather Impact**: Precipitation significantly reduces daily trip counts
- **Trip Duration**: Follows a log-normal distribution, typical for transportation data
- **User Demographics**: Members dominate usage, with male users being the majority

### Technical Achievements:

- Successfully created dual-axis time series visualization
- Implemented kernel density estimation for curve fitting
- Used subplot architecture for multi-panel comparisons
- Applied consistent styling and professional formatting